In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import numpy as np
import pandas as pd
import re, eli5, scipy, pickle
from sklearn.metrics import accuracy_score
import lightgbm as lgb
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.linear_model import LogisticRegression
from textblob import Word, TextBlob
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [2]:
fixed_skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [3]:
train = fetch_20newsgroups()
test = fetch_20newsgroups(subset="test")
X_train, y_train = train['data'], train['target']
X_test, y_test = test['data'], test['target']

In [37]:
tfidfvec = TfidfVectorizer(stop_words='english')
tfidf_train_bag = tfidfvec.fit_transform(X_train)
tfidf_test_bag = tfidfvec.transform(X_test)

In [38]:
logreg = LogisticRegression()
logreg.fit(tfidf_train_bag, y_train)

LogisticRegression()

In [39]:
best_weights = list(zip(np.abs(logreg.coef_).sum(axis=0), tfidfvec.get_feature_names_out()))
best_weights.sort(key = lambda x: x[0], reverse=True)

In [55]:
selected_columns = [x[1] for x in best_weights[:80_000]]

In [56]:
tfidfvec = TfidfVectorizer(vocabulary=selected_columns)
tfidf_train_bag = tfidfvec.fit_transform(X_train)
tfidf_test_bag = tfidfvec.transform(X_test)

In [97]:
logreg = LogisticRegression(max_iter=10_000, multi_class='multinomial', class_weight='balanced', C=4.0) #ovr
logreg.fit(tfidf_train_bag, y_train)

LogisticRegression(C=4.0, class_weight='balanced', max_iter=10000,
                   multi_class='multinomial')

In [99]:
preds = logreg.predict(tfidf_test_bag)

In [100]:
accuracy_score(y_test, preds)

0.8425385023898035